<h1 style="color: #00BFFF;">Web Scraping REAL</h1>

![let's go](https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGllM2RkeHVmM3diZ2Mzdzhydm00MjFrbW8zMWgwNHEzbTl5eGEyMiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/fJKG1UTK7k64w/giphy.gif)

# Web Scraping de libros (Books to Scrape) – Versión para alumnos

Objetivo de esta intro:
1. Descargar el HTML de la **primera página** de libros.
2. Extraer **títulos** y **precios** de cada libro.
3. Crear un **DataFrame de pandas** con esos datos (y dejar el precio como número).
4. (Opcional) Scraping de **múltiples páginas** con paginación.


## Código de Conducta
El fichero **robots.txt** es un archivo de texto que los sitios web colocan en su raíz (por ejemplo: https://example.com/robots.txt) y que sirve para indicar a los robots o crawlers (como Googlebot, Bingbot o scrapers) qué partes del sitio pueden o no pueden visitar.

- `https://books.toscrape.com/robots.txt`
- `https://openlibrary.org/robots.txt`

## 1) Preparación e importación de librerías

In [143]:
# Librería estándar
import re
import time
import os

# Librerías de terceros
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [144]:
# Headers para las peticiones HTTP
HEADERS = {
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": (
        "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/51.0.2704.64 Safari/537.36"
    ),
    "Accept": (
        "text/html,application/xhtml+xml,application/xml;q=0.9,"
        "image/webp,image/apng,*/*;q=0.8,"
        "application/signed-exchange;v=b3;q=0.9"
    ),
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
}

In [145]:
# ----------- Función para extraer detalle de cada libro por página individual -----------
def get_book_details(book_url: str) -> dict:
    """
    Obtiene información detallada de un libro desde su página individual:
    - stock exacto
    - categoría

    Parámetros
    ----------
    book_url : str
        URL de la página del libro.

    Retorna
    -------
    dict
        {
            "stock": int | None,
            "categoria": str | None
        }
    """
    if not book_url.startswith("http"):
        book_url = "http://books.toscrape.com/catalogue/" + book_url

    try:
        response = requests.get(book_url, headers=HEADERS)
        response.encoding = "utf-8"
        if response.status_code != 200:
            return {"stock": None, "categoria": None}

        soup = BeautifulSoup(response.text, "html.parser")

        # Stock
        availability_text = soup.find("p", class_="instock availability").get_text(strip=True)
        match = re.search(r"\((\d+) available\)", availability_text)
        stock = int(match.group(1)) if match else None

        # Categoría
        breadcrumb = soup.find("ul", class_="breadcrumb")
        categoria = None
        if breadcrumb:
            li_tags = breadcrumb.find_all("li")
            if len(li_tags) >= 3:
                # categoría real del libro
                categoria = li_tags[-2].get_text(strip=True)

        return {"stock": stock, "categoria": categoria}

    except Exception as e:
        print(f"Error al obtener detalles de {book_url}: {e}")
        return {"stock": None, "categoria": None}

    finally:
        time.sleep(0.5)

In [ ]:
# ----------- Función para extraer información de una sola página -----------
def scrape_page(url: str) -> list[dict]:
    """
    Extrae la información de todos los libros en una página de 'Books to Scrape'.

    Parámetros
    ----------
    url : str
        URL de la página que se desea scrapear.

    Devuelve
    --------
    list[dict]
        Lista de diccionarios, donde cada diccionario contiene:
        - "titulo": str, título del libro.
        - "precio": float, precio en libras (£).
        - "disponible": bool, True si está en stock.
        - "stock": int | None
        - "categoria": str | None
        - "estrellas": int, número de estrellas (1 a 5).

    Notas
    -----
    - Se fuerza la codificación a UTF-8 para evitar errores de caracteres.
    - El stock y categoria exacto está disponible en la web individual.
    """
    response = requests.get(url, headers=HEADERS)
    response.encoding = "utf-8"  # Forzar codificación UTF-8
    if response.status_code != 200:
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    books_data = []
    star_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    for book in books:
        title = book.h3.a["title"].strip()
        title = title.encode("utf-8").decode("utf-8")

        price_text = book.find("p", class_="price_color").get_text()
        price = float(price_text.replace("£", ""))

        availability_text = book.find("p", class_="instock availability").get_text(strip=True)
        disponible = "In stock" in availability_text

        # Obtener stock y categoría desde la página individual
        book_href = book.h3.a["href"]
        details = get_book_details(book_href)
        stock = details["stock"]
        categoria = details["categoria"]

        # Extraer número de estrellas
        star_tag = book.find("p", class_="star-rating")
        stars_class = star_tag["class"][1] if star_tag and len(star_tag["class"]) > 1 else "Zero"
        estrellas = star_map.get(stars_class, 0)

        books_data.append(
            {
                "titulo": title,
                "precio": price,
                "disponible": disponible,
                "stock": stock,  # Se puede incluir si se desea
                "categoria": categoria,
                "estrellas": estrellas,
            }
        )

    return books_data

In [147]:
# ----------- Función para obtener el número total de páginas -----------
def get_total_pages(base_url: str) -> int:
    """
    Obtiene el número total de páginas disponibles en el catálogo.

    Parámetros
    ----------
    base_url : str
        URL de la primera página del catálogo 
        (ejemplo: "http://books.toscrape.com/catalogue/page-1.html").

    Retorna
    -------
    int
        Número total de páginas encontradas en el catálogo.
        Si no se detecta el elemento, retorna 1 por defecto.
    """
    response = requests.get(base_url, headers=HEADERS)
    response.encoding = "utf-8"
    soup = BeautifulSoup(response.text, "html.parser")
    page_info = soup.find("li", class_="current")

    if page_info:
        match = re.search(r"Page \d+ of (\d+)", page_info.text.strip())
        if match:
            return int(match.group(1))

    return 1

In [ ]:
# ----------- Función para extraer información de todas las páginas dinámicamente -----------
def scrape_all_books(base_url_pattern: str) -> pd.DataFrame:
    """
    Recorre todas las páginas del catálogo dinámicamente y devuelve un DataFrame.

    Parámetros
    ----------
    base_url_pattern : str
        Patrón de URL que debe incluir un marcador `{}` para el número de página.
        Ejemplo: "http://books.toscrape.com/catalogue/page-{}.html".

    Retorna
    -------
    pd.DataFrame
        DataFrame con la información de todos los libros encontrados en el catálogo.
        Incluye título, precio, disponibilidad, stock, categoria y número de estrellas.
    """
    first_page_url = base_url_pattern.format(1)
    total_pages = get_total_pages(first_page_url)
    print(f"Se detectaron {total_pages} páginas en total.")

    all_books = []
    for page in range(1, total_pages + 1):
        url = base_url_pattern.format(page)
        print(f"Scrapeando página {page}...")
        all_books.extend(scrape_page(url))

    return pd.DataFrame(all_books)

In [ ]:
# ----------- Función para guardar resultados -----------
def save_results(
    df: pd.DataFrame,
    folder_path: str = ".",
    file_name: str = "books",
    save_csv: bool = True,
    save_parquet: bool = False # Instalar dependencias
) -> None:
    """
    Guarda el DataFrame en uno o varios formatos (CSV y/o Parquet) en la carpeta especificada.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame a guardar en archivo(s).
    folder_path : str, opcional
        Ruta de la carpeta donde se guardarán los archivos. 
        Por defecto, la carpeta actual `"."`.
    file_name : str, opcional
        Nombre base del archivo sin extensión. 
        Por defecto `"books"`.
    save_csv : bool, opcional
        Si es True, guarda el archivo en formato CSV. 
        Por defecto True.
    save_parquet : bool, opcional
        Si es True, guarda el archivo en formato Parquet. 
        Por defecto False.

    Retorna
    -------
    None
        Solo guarda los archivos en disco.
    """
    # Crear carpeta si no existe
    os.makedirs(folder_path, exist_ok=True)

    # Guardar como CSV
    if save_csv:
        csv_file = os.path.join(folder_path, f"{file_name}.csv")
        df.to_csv(csv_file, index=False, encoding="utf-8")
        print(f"CSV guardado en: {csv_file}")

    # Guardar como Parquet
    if save_parquet:
        parquet_file = os.path.join(folder_path, f"{file_name}.parquet")
        df.to_parquet(parquet_file, index=False)
        print(f"Parquet guardado en: {parquet_file}")

In [150]:
# ------------------- Funciones de prueba en Notebook -------------------

def test_get_book_details(book_url: str):
    """Prueba la función get_book_details() sobre un libro individual."""
    print("Test: get_book_details()")
    details = get_book_details(book_url)
    assert isinstance(details, dict), "Debe retornar un diccionario"
    assert "stock" in details and "categoria" in details, "Faltan claves en details"
    if details["stock"] is not None:
        assert isinstance(details["stock"], int) and details["stock"] >= 0, "Stock debe ser int >= 0"
    if details["categoria"] is not None:
        assert isinstance(details["categoria"], str) and len(details["categoria"]) > 0, "Categoria debe ser string no vacío"
    print("get_book_details() pasó correctamente.\n")


def test_scrape_page(page_url: str, check_first_n: int = 3):
    """Prueba la función scrape_page() sobre una sola página."""
    print("Test: scrape_page()")
    books = scrape_page(page_url)
    assert isinstance(books, list) and len(books) > 0, "Debe retornar lista con al menos un libro"
    expected_keys = ["titulo", "precio", "disponible", "stock", "estrellas", "categoria"]
    for i, book in enumerate(books[:check_first_n]):  # Solo revisar los primeros n libros
        for key in expected_keys:
            assert key in book, f"Falta clave {key} en libro {i+1}"
        assert isinstance(book["titulo"], str)
        assert isinstance(book["precio"], float) and book["precio"] > 0
        assert isinstance(book["disponible"], bool)
        if book["stock"] is not None:
            assert isinstance(book["stock"], int) and book["stock"] >= 0
        assert isinstance(book["estrellas"], int) and 0 <= book["estrellas"] <= 5
        if book["categoria"] is not None:
            assert isinstance(book["categoria"], str)
    print(f"scrape_page() pasó correctamente para los primeros {check_first_n} libros.\n")


def test_scrape_multiple_pages(base_url_pattern: str, max_pages: int = 5):
    """Prueba la función scrape_all_books() sobre un número limitado de páginas."""
    print(f"Test: scrape_all_books() (máx. {max_pages} páginas)")
    all_books = []
    for page in range(1, max_pages + 1):
        url = base_url_pattern.format(page)
        books = scrape_page(url)
        all_books.extend(books)
    import pandas as pd
    df_books = pd.DataFrame(all_books)
    expected_keys = ["titulo", "precio", "disponible", "stock", "estrellas", "categoria"]
    assert not df_books.empty, "El DataFrame no debe estar vacío"
    for col in expected_keys:
        assert col in df_books.columns, f"Falta columna {col} en DataFrame"
    print(f"scrape_all_books() pasó correctamente para las primeras {max_pages} páginas.\n")
    return df_books


def run_all_tests():
    """Ejecuta todas las pruebas secuencialmente."""
    print("Iniciando todas las pruebas...\n")
    # Test 1: libro individual
    test_get_book_details("http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html")
    # Test 2: una página
    test_scrape_page("http://books.toscrape.com/catalogue/page-1.html")
    # Test 3: varias páginas (limitadas a 5)
    base_url_pattern = "http://books.toscrape.com/catalogue/page-{}.html"
    df = test_scrape_multiple_pages(base_url_pattern, max_pages=5)
    print("Todas las pruebas pasaron correctamente.......\n")
    return df


# ------------------- Ejecutar todas las pruebas -------------------
df_books = run_all_tests()

Iniciando todas las pruebas...

Test: get_book_details()
get_book_details() pasó correctamente.

Test: scrape_page()
scrape_page() pasó correctamente para los primeros 3 libros.

Test: scrape_all_books() (máx. 5 páginas)
scrape_all_books() pasó correctamente para las primeras 5 páginas.

Todas las pruebas pasaron correctamente.......



In [151]:
# URL y paths
URL_PATTERN = "http://books.toscrape.com/catalogue/page-{}.html"

SAVE_PATH = "/home/neo/PROJECTS/IronHack_Esp_Big_Data/08_dia/exercises"
FILE_NAME = "books_catalog"

#-----------------------------------------------------------------------


# Definir URL base dinámica
base_url_pattern = URL_PATTERN

# Scraping de todas las páginas
df_books = scrape_all_books(base_url_pattern)

# Guardar en carpeta específica con nombre de archivo personalizado
save_results(df_books, folder_path=SAVE_PATH, file_name=FILE_NAME)

Se detectaron 50 páginas en total.
Scrapeando página 1...
Scrapeando página 2...
Scrapeando página 3...
Scrapeando página 4...
Scrapeando página 5...
Scrapeando página 6...
Scrapeando página 7...
Scrapeando página 8...
Scrapeando página 9...
Scrapeando página 10...
Scrapeando página 11...
Scrapeando página 12...
Scrapeando página 13...
Scrapeando página 14...
Scrapeando página 15...
Scrapeando página 16...
Scrapeando página 17...
Scrapeando página 18...
Scrapeando página 19...
Scrapeando página 20...
Scrapeando página 21...
Scrapeando página 22...
Scrapeando página 23...
Scrapeando página 24...
Scrapeando página 25...
Scrapeando página 26...
Scrapeando página 27...
Scrapeando página 28...
Scrapeando página 29...
Scrapeando página 30...
Scrapeando página 31...
Scrapeando página 32...
Scrapeando página 33...
Scrapeando página 34...
Scrapeando página 35...
Scrapeando página 36...
Scrapeando página 37...
Scrapeando página 38...
Scrapeando página 39...
Scrapeando página 40...
Scrapeando pág

# EDA

In [152]:
# Mostrar los primeros registros
df_books.head()

,titulo,precio,disponible,stock,categoria,estrellas
0,A Light in the Attic,51.77,True,22,Poetry,3
1,Tipping the Velvet,53.74,True,20,Historical Fiction,1
2,Soumission,50.10,True,20,Fiction,1
3,Sharp Objects,47.82,True,20,Mystery,4
4,Sapiens: A Brief History of Humankind,54.23,True,20,History,5


In [153]:
# Información general
df_books.info()

# Número total de libros scrapeados
print(f"\nTotal de libros scrapeados: {len(df_books)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   titulo      1000 non-null   object 
 1   precio      1000 non-null   float64
 2   disponible  1000 non-null   bool   
 3   stock       1000 non-null   int64  
 4   categoria   1000 non-null   object 
 5   estrellas   1000 non-null   int64  
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 40.2+ KB

Total de libros scrapeados: 1000
